In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load and Inspect Data
# Load the cryptocurrency market data into a Pandas DataFrame

df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display the first 10 rows of the dataset

df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics for initial exploration

df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Visualize the raw data trends

df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Normalize the dataset using StandardScaler
scaler = StandardScaler()
df_market_data_scaled = pd.DataFrame(
    scaler.fit_transform(df_market_data), 
    columns=df_market_data.columns, 
    index=df_market_data.index
)

# Display the first five rows of the normalized DataFrame
print(df_market_data_scaled.head())


              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash                     1.760610   

---

### Find the Optimal Number of Clusters (Elbow Method)


In [6]:
# Define range for k-values (from 1 to 11)
k_values = list(range(1, 12))

# Store inertia values for different k-values
inertia = []

# Compute inertia for different cluster sizes
for k in k_values:
    model = KMeans(n_clusters=k, random_state=0)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)


C:\Users\eek_e\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\eek_e\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\eek_e\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\eek_e\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Window

In [7]:
# Create a dictionary for plotting the Elbow Curve
elbow_data = {"k": k_values, "inertia": inertia}
elbow_df = pd.DataFrame(elbow_data)


In [8]:
# Plot the Elbow Curve
elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve", width=800, height=400)


:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 5

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [20]:
# Select the best k-value from the elbow curve manually (k = 5)
b_k = 5

# Initialize and fit K-Means model
model = KMeans(n_clusters=b_k, random_state=0)
df_market_data_scaled["Cluster"] = model.fit_predict(df_market_data_scaled)


C:\Users\eek_e\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [21]:
# Display the first few rows of the clustered DataFrame
print(df_market_data_scaled.head())

              price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash                     1.760610   

In [22]:
# Create a scatter plot of the clusters
df_market_data_scaled.hvplot.scatter(
    x="price_change_percentage_24h", 
    y="price_change_percentage_7d", 
    by="Cluster", 
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters - K-Means"
)


:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [23]:
# Reduce dimensions using PCA (3 principal components)
pca = PCA(n_components=3)
pca_transformed = pca.fit_transform(df_market_data_scaled.drop(columns=["Cluster"]))


In [24]:

# Create a new DataFrame with PCA-reduced data
df_market_data_pca = pd.DataFrame(
    pca_transformed, 
    columns=["PC1", "PC2", "PC3"], 
    index=df_market_data_scaled.index
)


In [25]:
# Display explained variance ratio for PCA components
print(f"Explained Variance: {sum(pca.explained_variance_ratio_):.2f}")

Explained Variance: 0.90


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is 0.90, which means that 90% of the variance in the original dataset is captured by these three components


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [26]:
# Compute inertia values using PCA-transformed data
inertia_pca = []
for k in k_values:
    model = KMeans(n_clusters=k, random_state=0)
    model.fit(df_market_data_pca)
    inertia_pca.append(model.inertia_)

C:\Users\eek_e\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\eek_e\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\eek_e\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\eek_e\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Window

In [27]:
# Plot Elbow Curve for PCA data
elbow_pca_data = {"k": k_values, "inertia": inertia_pca}
elbow_pca_df = pd.DataFrame(elbow_pca_data)
elbow_pca_df.hvplot.line(x="k", y="inertia", title="Elbow Curve (PCA Data)", width=800, height=400)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**The optimal value for k using the PCA-transformed data is 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Yes, it differs by 1, as the best k for the original scaled data was 5, while for PCA it is 4.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [29]:
# Select the best k-value for PCA Data (k = 4)
b_k_pca = 4

# Initialize K-Means and fit model
model_pca = KMeans(n_clusters=b_k_pca, random_state=0)
df_market_data_pca["Cluster"] = model_pca.fit_predict(df_market_data_pca)


C:\Users\eek_e\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [30]:
# Create a scatter plot for PCA-based clusters
df_market_data_pca.hvplot.scatter(
    x="PC1", y="PC2", by="Cluster", 
    hover_cols=["coin_id"], 
    title="Cryptocurrency Clusters - PCA Optimized"
)


:NdOverlay   [Cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In [31]:
# Composite plot comparing original Elbow Curve with PCA-transformed Elbow Curve
(elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve (Original Data)") + 
 elbow_pca_df.hvplot.line(x="k", y="inertia", title="Elbow Curve (PCA Data)"))



:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [32]:
# Composite plot comparing original clustering with PCA clustering
(df_market_data_scaled.hvplot.scatter(x="price_change_percentage_24h", y="price_change_percentage_7d", by="Cluster") + 
 df_market_data_pca.hvplot.scatter(x="PC1", y="PC2", by="Cluster"))


:Layout
   .NdOverlay.I  :NdOverlay   [Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)
   .NdOverlay.II :NdOverlay   [Cluster]
      :Scatter   [PC1]   (PC2)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features with PCA simplifies clustering by focusing on key variance, leading to clearer, well-separated clusters. After reducing to three principal components, the clusters became more distinct and interpretable, improving K-Means results.